In [41]:
import warnings


%load_ext autoreload


%autoreload 2

warnings.filterwarnings('ignore')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import pandas as pd 
from config import get_embedding, get_chat

In [40]:
CRITICS = {
    "REUBEN BARON": "https://www.rottentomatoes.com/critics/reuben-baron/movies",
    "RON SEOUL-OH": "https://www.rottentomatoes.com/critics/ron-seoul-oh/movies", 
    "LINDA MARRIC": "https://www.rottentomatoes.com/critics/linda-marric/movies", 
    "ISHITA SENGUPTA": "https://www.rottentomatoes.com/critics/ishita-sengupta/movies",   
    "GANESH AAGLAVE": "https://www.rottentomatoes.com/critics/ganesh-aaglave/movies",
    "DANA KENNEDY": "https://www.rottentomatoes.com/critics/dana-kennedy/movies",
    "NATALIA KEOGAN": "https://www.rottentomatoes.com/critics/natalia-keogan/movies"
}

In [11]:
data = pd.DataFrame()
for critic, url in CRITICS.items():
    tmp_df = pd.read_html(url)[0]
    tmp_df['Critic'] = critic
    data = data.append(tmp_df)

In [13]:
def normalize_rating(x):
    try: 
        if '/' in x or x == 0:
            return eval(x) * 100
    except:
        return None
    
data['Rating'] = data.Rating.apply(normalize_rating)
data = data.dropna(subset=['Rating'])

In [14]:
# Define bins and labels
bins = [-1, 50, 70, 90, 100]  # We use -1 as the start to include 0 in the first bin
labels = ['bad', 'neutral', 'positive', 'very positive']

# Use pd.cut to classify 'score' and create 'rating' column
data['Sentiment'] = pd.cut(data['Rating'], bins=bins, labels=labels, right=True)
data[['Title', 'Year']] = data['Title | Year'].str.extract(r'(.+)\s\((\d{4})\)')
data['critic_count'] = data.groupby('Critic')['Critic'].transform('count')
data.drop(columns=['Title | Year', 'T-Meter'], inplace=True)
data

,Rating,Review,Critic,Sentiment,Title,Year,critic_count
0,100.0,It's the exact sort of movie you'll want to te...,REUBEN BARON,very positive,Hit Man,2023,41
1,60.0,The existence of The Peasants is a net good fo...,REUBEN BARON,neutral,The Peasants,2023,41
2,75.0,"Fortunately, it's still a really funny movie w...",REUBEN BARON,positive,Next Goal Wins,2023,41
3,90.0,"Even if it's not Miyazaki's swan song, this mo...",REUBEN BARON,positive,The Boy and the Heron,2023,41
4,80.0,[Bottoms] further establishes Seligman as a di...,REUBEN BARON,positive,Bottoms,2023,41
...,...,...,...,...,...,...,...
45,80.0,"Teeming with death, denial and damning governm...",NATALIA KEOGAN,positive,The Sadness,2021,49
46,68.0,Along for the Ride is a warm cup of duck soup ...,NATALIA KEOGAN,neutral,Along for the Ride,2022,49
47,83.0,In Front of Your Face beautifully maximizes th...,NATALIA KEOGAN,positive,In Front of Your Face,2021,49
48,82.0,Racked with emotional tension and visceral tur...,NATALIA KEOGAN,positive,Happening,2021,49


In [15]:
filter_mask = (data.critic_count >= 5)
data = data[filter_mask]

In [16]:
data

,Rating,Review,Critic,Sentiment,Title,Year,critic_count
0,100.0,It's the exact sort of movie you'll want to te...,REUBEN BARON,very positive,Hit Man,2023,41
1,60.0,The existence of The Peasants is a net good fo...,REUBEN BARON,neutral,The Peasants,2023,41
2,75.0,"Fortunately, it's still a really funny movie w...",REUBEN BARON,positive,Next Goal Wins,2023,41
3,90.0,"Even if it's not Miyazaki's swan song, this mo...",REUBEN BARON,positive,The Boy and the Heron,2023,41
4,80.0,[Bottoms] further establishes Seligman as a di...,REUBEN BARON,positive,Bottoms,2023,41
...,...,...,...,...,...,...,...
45,80.0,"Teeming with death, denial and damning governm...",NATALIA KEOGAN,positive,The Sadness,2021,49
46,68.0,Along for the Ride is a warm cup of duck soup ...,NATALIA KEOGAN,neutral,Along for the Ride,2022,49
47,83.0,In Front of Your Face beautifully maximizes th...,NATALIA KEOGAN,positive,In Front of Your Face,2021,49
48,82.0,Racked with emotional tension and visceral tur...,NATALIA KEOGAN,positive,Happening,2021,49


In [17]:
def augment_context(row):
    return f"""
    CRITIC: {row['Critic']}
    RATING: {row['Rating']}
    SENTIMENT: {row['Sentiment']}
    MOVIE: {row['Title']}
    YEAR: {row['Year']}
    REVIEW: {row['Review']}
    """
    
data["context"] = data.apply(augment_context, axis=1)

In [18]:
# data["embeddings"] = data["context"].apply(get_embedding)
# data.reset_index(inplace=True, drop=True)

In [21]:
import chromadb
client = chromadb.PersistentClient()
collection = client.get_or_create_collection("rotten_tomatoes_reviews")

In [43]:
%pre-commit install

UsageError: Line magic function `%pre-commit` not found.


In [23]:
# embeddings = data.embeddings.values.tolist()
# ids = data.index.astype(str).values.tolist()
# documents = data.context.values.tolist()
# metadata = data[['Rating', 'Review', 'Critic', 'Sentiment', 'Title', 'Year', 'critic_count', 'context']].to_dict(orient='records')

In [24]:
# collection.add(
#     embeddings=embeddings,
#     metadatas=metadata,
#     documents=documents,
#     ids=ids,
# )

In [25]:
query_result = collection.query(query_embeddings=[0]*1536, n_results=5, include=["documents", "distances"])

In [26]:
prompt_template = """
HERE IS MY TASK: {prompt}

==== EXAMPLE OUTPUT DELIMITED BY TRIPLE QUOTES ====
{few_shot_examples}
===== END OF EXAMPLE OUTPUT DELIMITED BY TRIPLE QUOTES =====

Mirror the example outputs given. Think step-by-step about how you would respond to the prompt. Mirrored Example Output Response:
"""

In [27]:
data.Critic.unique()

array(['REUBEN BARON', 'RON SEOUL-OH', 'LINDA MARRIC', 'GANESH AAGLAVE',
       'DANA KENNEDY', 'NATALIA KEOGAN'], dtype=object)

In [28]:
CRITIC = "NATALIA KEOGAN"
SENTIMENT = "positive"
MOVIE = "Whispers of the Nebula"
MOVIE_DESCRIPTION = """In a distant future, a renowned astronaut discovers an ancient alien civilization within a mesmerizing nebula, only to realize that the echoes of their history hold the key to saving Earth from an impending cosmic disaster."""
REVIEW_ROUGH_DRAFT = """
- Cool space adventures meet old alien secrets.
- Makes you think about Earth's place in space.
- Mixes space tales with ancient puzzles.
"""

initial_prompt = f"""
Write a {SENTIMENT} review about a {MOVIE}. I want you to write the review in the style of {CRITIC}.

The movie description is: {MOVIE_DESCRIPTION}

Here are things I want to hit on in my review: {REVIEW_ROUGH_DRAFT}
"""	

In [29]:
prompt_embedding = get_embedding(initial_prompt)
query_result = collection.query(prompt_embedding, n_results=10, include=["documents", "distances", "metadatas"], where={"Critic": CRITIC})

few_shot_examples = """ """
for i, result in enumerate(query_result["metadatas"][0]):
    review = f"""
    '''
    Example {i + 1}: {result["Review"]}
    '''
    """
    few_shot_examples += review

prompt = prompt_template.format(prompt=initial_prompt, few_shot_examples=few_shot_examples)
print(prompt)


HERE IS MY TASK: 
Write a positive review about a Whispers of the Nebula. I want you to write the review in the style of NATALIA KEOGAN.

The movie description is: In a distant future, a renowned astronaut discovers an ancient alien civilization within a mesmerizing nebula, only to realize that the echoes of their history hold the key to saving Earth from an impending cosmic disaster.

Here are things I want to hit on in my review: 
- Cool space adventures meet old alien secrets.
- Makes you think about Earth's place in space.
- Mixes space tales with ancient puzzles.



==== EXAMPLE OUTPUT DELIMITED BY TRIPLE QUOTES ====
 
    '''
    Example 1: Eslyn’s directorial turn is confident and sharp, born from an extensive collaboration with Duplass that coaxes genuine curiosity from the ordinary crevices of human perspective and experience. - Paste Magazine  Read More  | Posted Jul 10, 2023
    '''
    
    '''
    Example 2: This heartfelt elegy, born from an encounter in Leaf’s past, is 

In [33]:
messages = [{"role": "user", "content": prompt}]
output = get_chat(messages, model="gpt-4")

In [32]:
print(output)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "'''\n\"Whispers of the Nebula\" is a thrilling cosmic journey that seamlessly blends cool space adventures with the enigma of ancient alien secrets. The director's vision is both audacious and meticulous, crafting a narrative that is as expansive as the universe it explores, yet as intimate as the human struggle for survival. - Paste Magazine  Read More  | Posted Sep 15, 2023\n'''\n\n'''\nThis film is a cerebral exploration of Earth's place in the cosmos, wrapped in a narrative that is as engaging as it is thought-provoking. The ancient puzzles that the protagonist must solve are not just a plot device, but a metaphor for humanity's quest for understanding and survival. - Paste Magazine  Read More  | Posted Sep 16, 2023\n'''\n\n'''\n\"Whispers of the Nebula\" is a testament to the power of storytelling, where the vastness of space serves as a backdrop for a tale that is deeply